# Exploring industry returns in the Fama-French data-set

The [49 Industry Portfolios](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_49_ind_port.html) data-set has a series of daily returns of 49 different industry types. Here, we look at some of their statistical properties.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
startDt <- as.Date("1990-01-01")

#value (market-cap) weighted
valueWtd <- famaFrench$MomentumDaily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

#equal weighted
equalWtd <- famaFrench$MomentumDaily() %>%
    filter(RET_TYPE == 'AEWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

In [3]:
lookbackDays <- 220 * 5

rrValueWtd <- rollapply(valueWtd, lookbackDays, Return.cumulative, by.column = F)
rrEqualWtd <- rollapply(equalWtd, lookbackDays, Return.cumulative, by.column = F)

rrValueWtd <- na.omit(rrValueWtd)
rrEqualWtd <- na.omit(rrEqualWtd)

print(head(rrValueWtd))
print(head(rrEqualWtd))

               [,1]        [,2]      [,3]      [,4]      [,5]      [,6]
1994-05-06 1.295753 -0.03668894 0.2000777 0.2620761 0.3428043 0.3670918
1994-05-09 1.231004 -0.07221744 0.1559846 0.2226555 0.3003139 0.3306055
1994-05-10 1.236360 -0.07425205 0.1579408 0.2308171 0.3019994 0.3481135
1994-05-11 1.232287 -0.07733569 0.1415276 0.2183684 0.2866327 0.3380944
1994-05-12 1.262496 -0.07326946 0.1543795 0.2274315 0.2904911 0.3563674
1994-05-13 1.243091 -0.06758286 0.1519676 0.2273090 0.2950060 0.3528527
                [,7]      [,8]      [,9]     [,10]
1994-05-06 0.2617704 0.5643377 0.4807484 0.7725134
1994-05-09 0.2330314 0.5375320 0.4412131 0.7156842
1994-05-10 0.2372165 0.5465989 0.4520428 0.7292680
1994-05-11 0.2332332 0.5455088 0.4543846 0.7289189
1994-05-12 0.2447831 0.5683057 0.4728875 0.7492262
1994-05-13 0.2506371 0.5694028 0.4610318 0.7285346
               [,1]     [,2]     [,3]     [,4]     [,5]     [,6]     [,7]
1994-05-06 3.261914 45.31315 3.055510 2.113199 1.924780 1.353790 

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)